<a href="https://colab.research.google.com/github/tiarayosianti/Spatial-SCINet/blob/main/Model_Spatial_SCINet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import modules

Import modules

In [ ]:
!python --version
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
import random
import numpy as np
import pandas as pd
from time import time
from math import sqrt
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
# from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from module.train_model import train_conv2d_scinet
from module.SpatialSCINet import Conv2D_SCINet

#Preprocess data

Perform data scaling

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/new_df_clean_pm10_meteorologi.csv')
data_x = dataset[['Tavg',	'RH_avg',	'ff_avg',	'RR']]
data_y_pm10 = dataset[['PM10']]

scaler_x = StandardScaler()
scaler_y = StandardScaler()

data_x_scaled = scaler_x.fit_transform(data_x)
data_y_scaled_pm10 = scaler_y.fit_transform(data_y_pm10)

data_x_scaled = pd.DataFrame(data_x_scaled, columns = ['Tavg',	'RH_avg',	'ff_avg',	'RR'])
data_y_scaled = pd.DataFrame(data_y_scaled_pm10, columns = ['PM10'])

data_scaled = dataset[['Tanggal','kota', 'Latitude', 'Longitude']]
data_scaled = pd.concat([data_scaled, data_y_scaled, data_x_scaled], axis=1)

print(data_scaled.shape)
data_scaled.head()

In [ ]:
data_x_scaled.max(), data_y_scaled.max(), data_x_scaled.min(), data_y_scaled.min()

In [ ]:
dataset.max(), dataset.min()

#Function


In [ ]:
# Creating 4-dimensional array
def create_dataset(Latitude, Longitude, WINDOW_SIZE, horizon):
    location_current = data_scaled.loc[((data_scaled['Latitude'] == Latitude) & (data_scaled['Longitude'] == Longitude))]
    variables = location_current[['PM10',	'Tavg',	'RH_avg',	'ff_avg',	'RR']]
    variables = np.array(variables)
    X, Y = [], []
    for i in range(len(variables) - WINDOW_SIZE - horizon):
        X.append(variables[i:(i + WINDOW_SIZE), 1:][::-1])
        Y.append(variables[(i + WINDOW_SIZE):(i + WINDOW_SIZE + horizon), 0]) # PM10 is in column index 0
    return X, Y


# Split the dataset for train, validation, and test data
def split_data(X_array, Y_array, train_frac, val_frac, test_frac, N_samples):
    train_X, val_X, test_X = X_array[:int(train_frac * N_samples), :], X_array[int(train_frac * N_samples):int((val_frac + train_frac) * N_samples), :], X_array[int((val_frac + train_frac) * N_samples):, :]
    train_y, val_y, test_y = Y_array[:int(train_frac * N_samples), :], Y_array[int(train_frac * N_samples):int((val_frac + train_frac) * N_samples), :], Y_array[int((val_frac + train_frac) * N_samples):, :]
    return train_X, val_X, test_X, train_y, val_y, test_y


# Visualiza the train loss vs validation loss
def learning_curve(history_model, model_name):
  plt.plot(history_model.history['loss'], label = 'train')
  plt.plot(history_model.history['val_loss'], label = 'val')
  plt.title('Train and Validation Loss: {}'.format(model_name))
  plt.legend()
  plt.show()


# Inverse transform the ground truth y and predicted value
def invers_trans(model_name, pred_y, true_y):
  inv_pred_y = []
  for yhat in pred_y:
    inv_yhat = yhat.reshape((yhat.shape[0]*yhat.shape[1], yhat.shape[2]))
    inv_yhat = scaler_y.inverse_transform(inv_yhat)
    inv_pred_y.append(inv_yhat)
  inv_true_y = true_y.reshape((true_y.shape[0]*true_y.shape[1], true_y.shape[2]))
  inv_true_y = scaler_y.inverse_transform(inv_true_y)
  return inv_pred_y, inv_true_y


# Evaluation metrics
def evaluation_metrics(y_actual, y_predict, model_name):
  em_summary = pd.DataFrame(columns = ['model_name', 'MAPE', 'MAE', 'RMSE'])
  for idx, y_predict_seq in enumerate(y_predict):
    new_row = {'model_name': model_name[idx],
            'MAPE': mean_absolute_percentage_error(y_actual, y_predict_seq)*100,
            'MAE': mean_absolute_error(y_actual, y_predict_seq),
            'RMSE': sqrt(mean_squared_error(y_actual, y_predict_seq))}
    em_summary.loc[idx] = new_row
  return em_summary


# Search the best model
def the_best_model(df):
    df['metrics'] = df[['RMSE', 'MAE', 'MAPE']].sum(axis=1)
    best_model = df[df['metrics'] == df['metrics'].min()]['model_name']
    return best_model.values.tolist()


# Check the level number that compatible with the window size
def check_input(comb_params):
  rule_1 = []
  rule_2 = []
  rule_all = []
  for i in range(len(comb_params)):
        # rule num 1
        if comb_params['input_len'][i] % 2**comb_params['num_levels'][i] == 0:
          rule_1.append('ok')
        else:
          rule_1.append('bad')
        # rule num 2
        if (comb_params['input_len'][i] / 2**comb_params['num_levels'][i]) % 2 == 0:
          rule_2.append('ok')
        else:
          rule_2.append('bad')
  df_rule = pd.DataFrame()
  df_rule['rule_1'] = rule_1
  df_rule['rule_2'] = rule_2
  for i in range(len(df_rule)):
        # all rules
        if df_rule['rule_1'][i] == 'ok' and df_rule['rule_2'][i] == 'ok':
          rule_all.append('use it!')
        else:
          rule_all.append('bad')
  df_rule['rule_all'] = rule_all
  df_rule2 = pd.concat([comb_params, df_rule], axis = 1)
  df_rule2 = df_rule2.sort_values(by=['rule_all', 'input_len'], ascending=[False,True])
  return df_rule2

the_param = {'input_len' :  [24, 30, 48], 'num_levels' : [2,	3,	4,	5]}
the_param_combinations = list(ParameterGrid(the_param))
df_param = pd.DataFrame(the_param_combinations)
check_result = check_input(df_param)
check_result

#Modelling

In [ ]:
# Creating 4D data shape
latitude = data_scaled['Latitude'].unique()
longitude = data_scaled['Longitude'].unique()
coords = list(zip(latitude, longitude))
print(coords)

df_X, df_Y = [], []
for lat, lon in tqdm(coords):
    a, b = create_dataset(Latitude = lat,	Longitude = lon, WINDOW_SIZE = 24, horizon = 24)
    df_X.append(a)
    df_Y.append(b)

df_X = np.moveaxis(df_X,0,-1)
df_Y = np.moveaxis(df_Y,0,-1)
print("\n \n X shape :", df_X.shape)
print(" Y shape :", df_Y.shape)


# Split the dataset
N_samples = df_X.shape[0]
train_frac = 0.6
val_frac = 0.2
test_frac = 0.2

train_size = int(N_samples * train_frac)
val_size = int(N_samples * val_frac)
test_size = N_samples - train_size - val_size
print("\nData train rows      :", train_size)
print("Data validation rows :", val_size)
print("Data test rows       :", test_size)

train_X, val_X, test_X, train_y, val_y, test_y = split_data(df_X, df_Y, train_frac, val_frac, test_frac, N_samples)
print("\nData shape")
print("train_X :", train_X.shape)
print("val_X   :", val_X.shape)
print("test_X  :", test_X.shape)
print("train_y :", train_y.shape)
print("val_y   :", val_y.shape)
print("test_y  :", test_y.shape)

##Hyperparameter tunning

## Tanh

We will conduct several experiments for the activation function on the Conv2D Block. In this case, for the first layer we will use Tanh. However, for the second layer we will try using Leaky ReLU, ELU and Tanh.

### 1. tanh_lrelu

In [ ]:
### Hyperparameter Tuning Combinations
np.random.seed(100)
tf.random.set_seed(100)

# define parameter grid
param_grid = {'batch_size' : [16, 32, 64],
              'learning_rate' : [0.0001, 0.0003, 0.0005, 0.0007, 0.0009],
              'hid_size' : [4, 8]}

# create parameter combinations
param_combinations = list(ParameterGrid(param_grid))
params_detail = pd.DataFrame(param_combinations)
params_detail

In [ ]:
# Build Model

models = []
history_models = []

for params in param_combinations:
    print(params)

    results = train_conv2d_scinet(
                        X_train = train_X,
                        y_train = train_y,
                        X_val = val_X,
                        y_val = val_y,
                        X_test = test_X,
                        y_test = test_y,
                        epochs = 20,
                        batch_size = params['batch_size'],
                        X_LEN = 24, # window size
                        Y_LEN = [24], # horizon
                        output_dim = [5], # locations
                        selected_columns = None,
                        hid_size = params['hid_size'],
                        num_levels = 2,
                        kernel = 5,
                        dropout = 0.5,
                        loss_weights = [1],
                        learning_rate = params['learning_rate'],
                        filters = [16, 8],
                        kernel_size = (7, 4), # 7 days, 4 variables
                        activation_func = ['tanh','leaky_relu'])

    models.append(results[0])
    history_models.append(results[1])

In [ ]:
# the list of model names
model_name = []
for i in range(len(models)):
  model_name.append('Model {}'.format(i))
model_name

In [ ]:
# plot history
for i in range(len(models)):
  learning_curve(history_models[i], model_name[i])

In [ ]:
# save model
for idx, model in enumerate(models):
  model.save_weights('/content/drive/MyDrive/Bobot/tanh_lrelu_{}.h5'.format(idx))

#### Predict validation data

In [ ]:
# Build model to load the weights
model_ = []
for params in param_combinations:
    print(params)
    model =  Conv2D_SCINet( output_len = [24],
                       output_dim = [5],
                       input_len = 24,
                       input_dim = train_X.shape[2] * train_X.shape[3],
                       x_features = train_X.shape[2],
                       locations = train_X.shape[3],
                       selected_columns = None,
                       hid_size = params['hid_size'],
                       num_levels = 2,
                       kernel = 5,
                       dropout = 0.5,
                       loss_weights = [1],
                       learning_rate = params['learning_rate'],
                       probabilistic = False,
                       filters = [16, 8],
                       kernel_size = (7, 4),
                       activation_func = ['tanh','leaky_relu'])
    model = model.build_model()
    model_.append(model)

In [ ]:
# predict data validation
yhats_val = []
for idx, model in enumerate(model_):
    model.load_weights('/content/drive/MyDrive/Bobot/tanh_lrelu_{}.h5'.format(idx))
    yhat = model.predict(val_X)
    yhats_val.append(yhat)

In [ ]:
# evaluation metrics
inv_yhat_val, inv_val_y = invers_trans(model_name, yhats_val, val_y)
eval_metrics_val = evaluation_metrics(inv_val_y, inv_yhat_val, model_name)

# the best model
print('The best model')
the_best_model(eval_metrics_val)

In [ ]:
eval_metrics_val

#### Predict test data

In [ ]:
# predict data test
yhats = []
for idx, model in enumerate(model_):
    model.load_weights('/content/drive/MyDrive/Bobot/tanh_lrelu_{}.h5'.format(idx))
    yhat = model.predict(test_X)
    yhats.append(yhat)

In [ ]:
# evaluation metrics
inv_yhat, inv_test_y = invers_trans(model_name, yhats, test_y)
eval_metrics = evaluation_metrics(inv_test_y, inv_yhat, model_name)

# the best model
print('The best model')
the_best_model(eval_metrics)

In [ ]:
eval_metrics

In [ ]:
params_detail

### 2. tanh_elu

In [ ]:
### Hyperparameter Tuning Combinations
np.random.seed(100)
tf.random.set_seed(100)

# define parameter grid
param_grid = {'batch_size' : [16, 32, 64],
              'learning_rate' : [0.0001, 0.0003, 0.0005, 0.0007, 0.0009],
              'hid_size' : [4, 8]}

# create parameter combinations
param_combinations = list(ParameterGrid(param_grid))
params_detail = pd.DataFrame(param_combinations)
params_detail

In [ ]:
# Build Model

models = []
history_models = []

for params in param_combinations:
    print(params)

    results = train_conv2d_scinet(
                        X_train = train_X,
                        y_train = train_y,
                        X_val = val_X,
                        y_val = val_y,
                        X_test = test_X,
                        y_test = test_y,
                        epochs = 20,
                        batch_size = params['batch_size'],
                        X_LEN = 24, # window size
                        Y_LEN = [24], # horizon
                        output_dim = [5], # locations
                        selected_columns = None,
                        hid_size = params['hid_size'],
                        num_levels = 2,
                        kernel = 5,
                        dropout = 0.5,
                        loss_weights = [1],
                        learning_rate = params['learning_rate'],
                        filters = [16, 8],
                        kernel_size = (7, 4), # 7 days, 4 variables
                        activation_func = ['tanh','elu'])

    models.append(results[0])
    history_models.append(results[1])

In [ ]:
# the list of model names
model_name = []
for i in range(len(models)):
  model_name.append('Model {}'.format(i))
model_name

In [ ]:
# plot history
for i in range(len(models)):
  learning_curve(history_models[i], model_name[i])

In [ ]:
# save model
for idx, model in enumerate(models):
  model.save_weights('/content/drive/MyDrive/Bobot/tanh_elu_{}.h5'.format(idx))

#### Predict validation data

In [ ]:
# Build model to load the weights
model_ = []
for params in param_combinations:
    print(params)
    model =  Conv2D_SCINet( output_len = [24],
                       output_dim = [5],
                       input_len = 24,
                       input_dim = train_X.shape[2] * train_X.shape[3],
                       x_features = train_X.shape[2],
                       locations = train_X.shape[3],
                       selected_columns = None,
                       hid_size = params['hid_size'],
                       num_levels = 2,
                       kernel = 5,
                       dropout = 0.5,
                       loss_weights = [1],
                       learning_rate = params['learning_rate'],
                       probabilistic = False,
                       filters = [16, 8],
                       kernel_size = (7, 4),
                       activation_func = ['tanh','elu'])
    model = model.build_model()
    model_.append(model)

In [ ]:
# predict data validation
yhats_val = []
for idx, model in enumerate(model_):
    model.load_weights('/content/drive/MyDrive/Bobot/tanh_elu_{}.h5'.format(idx))
    yhat = model.predict(val_X)
    yhats_val.append(yhat)

In [ ]:
# evaluation metrics
inv_yhat_val, inv_val_y = invers_trans(model_name, yhats_val, val_y)
eval_metrics_val = evaluation_metrics(inv_val_y, inv_yhat_val, model_name)

# the best model
print('The best model')
the_best_model(eval_metrics_val)

In [ ]:
eval_metrics_val

#### Predict test data

In [ ]:
# predict data test
yhats = []
for idx, model in enumerate(models):
    model.load_weights('/content/drive/MyDrive/Bobot/tanh_elu_{}.h5'.format(idx))
    yhat = model.predict(test_X)
    yhats.append(yhat)

In [ ]:
# evaluation metrics
inv_yhat, inv_test_y = invers_trans(model_name, yhats, test_y)
eval_metrics = evaluation_metrics(inv_test_y, inv_yhat, model_name)

In [ ]:
# the best model
print('The best model')
the_best_model(eval_metrics)

In [ ]:
eval_metrics

In [ ]:
params_detail

### 3. tanh_tanh

In [ ]:
### Hyperparameter Tuning Combinations
np.random.seed(100)
tf.random.set_seed(100)

# define parameter grid
param_grid = {'batch_size' : [16, 32, 64],
              'learning_rate' : [0.0001, 0.0003, 0.0005, 0.0007, 0.0009],
              'hid_size' : [4, 8]}

# create parameter combinations
param_combinations = list(ParameterGrid(param_grid))
params_detail = pd.DataFrame(param_combinations)
params_detail

In [ ]:
# Build Model

models = []
history_models = []

for params in param_combinations:
    print(params)

    results = train_conv2d_scinet(
                        X_train = train_X,
                        y_train = train_y,
                        X_val = val_X,
                        y_val = val_y,
                        X_test = test_X,
                        y_test = test_y,
                        epochs = 20,
                        batch_size = params['batch_size'],
                        X_LEN = 24, # window size
                        Y_LEN = [24], # horizon
                        output_dim = [5], # locations
                        selected_columns = None,
                        hid_size = params['hid_size'],
                        num_levels = 2,
                        kernel = 5,
                        dropout = 0.5,
                        loss_weights = [1],
                        learning_rate = params['learning_rate'],
                        filters = [16, 8],
                        kernel_size = (7, 4), # 7 days, 4 variables
                        activation_func = ['tanh','tanh'])

    models.append(results[0])
    history_models.append(results[1])

In [ ]:
# the list of model names
model_name = []
for i in range(len(models)):
  model_name.append('Model {}'.format(i))
model_name

In [ ]:
# plot history
for i in range(len(models)):
  learning_curve(history_models[i], model_name[i])

In [ ]:
# save model
for idx, model in enumerate(models):
  model.save_weights('/content/drive/MyDrive/Bobot/tanh_tanh_{}.h5'.format(idx))

#### Predict validation data

In [ ]:
# Build model to load the weights
model_ = []
for params in param_combinations:
    print(params)
    model =  Conv2D_SCINet( output_len = [24],
                       output_dim = [5],
                       input_len = 24,
                       input_dim = train_X.shape[2] * train_X.shape[3],
                       x_features = train_X.shape[2],
                       locations = train_X.shape[3],
                       selected_columns = None,
                       hid_size = params['hid_size'],
                       num_levels = 2,
                       kernel = 5,
                       dropout = 0.5,
                       loss_weights = [1],
                       learning_rate = params['learning_rate'],
                       probabilistic = False,
                       filters = [16, 8],
                       kernel_size = (7, 4),
                       activation_func = ['tanh','tanh'])
    model = model.build_model()
    model_.append(model)

In [ ]:
# predict data validation
yhats_val = []
for idx, model in enumerate(model_):
    model.load_weights('/content/drive/MyDrive/Bobot/tanh_tanh_{}.h5'.format(idx))
    yhat = model.predict(val_X)
    yhats_val.append(yhat)

In [ ]:
# evaluation metrics
inv_yhat_val, inv_val_y = invers_trans(model_name, yhats_val, val_y)
eval_metrics_val = evaluation_metrics(inv_val_y, inv_yhat_val, model_name)

# the best model
print('The best model')
the_best_model(eval_metrics_val)

In [ ]:
eval_metrics_val

#### Predict test data

In [ ]:
# predict data test
yhats = []
for idx, model in enumerate(model_):
    model.load_weights('/content/drive/MyDrive/Bobot/tanh_tanh_{}.h5'.format(idx))
    yhat = model.predict(test_X)
    yhats.append(yhat)

In [ ]:
# evaluation metrics
inv_yhat, inv_test_y = invers_trans(model_name, yhats, test_y)
eval_metrics = evaluation_metrics(inv_test_y, inv_yhat, model_name)

In [ ]:
# the best model
print('The best model')
the_best_model(eval_metrics)

In [ ]:
eval_metrics

In [ ]:
params_detail

#End of notebook